<a href="https://colab.research.google.com/github/vincentjordan27/Named-Entity-Recognition-BILSTM-CRF/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import sys
import csv

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install torchtext==0.6.0

import time
import torch
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from spacy.lang.id import Indonesian

     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 34.5 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [8]:
maxInt = sys.maxsize

while True:
   
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt / 10)

dataset = '/content/drive/My Drive/Colab Notebooks/SINGGALANG.tsv'
data_df = pd.read_csv(dataset, sep='\t', error_bad_lines=False, header=None, engine="python", names=['word', 'tag'],
                      quoting=csv.QUOTE_NONE)
#print(data_df.head())


In [9]:
class Corpus(object):

  def __init__(self, input, min_word_freq, batch_size):
    self.word_field = Field(lower=True)
    self.tag_field= Field(unk_token=None)

    self.train_dataset, self.val_dataset, self.test_dataset = SequenceTaggungDataset.split(
        path=input,
        train="train.tsv",
        validation="val.tsv",
        test="test.tsv",
        fields=(("word", self.word_field), ("tag", self.tag_field))
    )

    self.word_field.build_vocab(self.train_dataset.word, min_freg=min_word_freg)
    self.tag_field.build_voab(self.train_dataset.tag)

    self.train_iter, self.val_iter, self.test_iter = BucketIterator.splits(
        datasets=(self.train_dataset, self.val_dataset, self.test_dataset),
        batch_size=batch_size
    )
    
    self.word_pad_idx = self.word_field.vocab.stoi[self.word_field.pad_token]
    self.tag_pad_idx = self.tag_field.vocab.stoi[self.tag_field.pad_token]
    

In [11]:
corpus = Corpus(
    input="dataset",
    min_word_freq=2,
    batch_size=60
)

NameError: ignored

In [12]:
wv_shape = corpus.wv_model.wv.vectors.shape


NameError: ignored